# hashed-ezkl

Here's an example leveraging EZKL whereby the inputs to the model, and the model params themselves, are hashed inside a circuit.

In this setup:
- the hashes are publicly known to the prover and verifier
- the hashes serve as "public inputs" (a.k.a instances) to the circuit

We leave the outputs of the model as public as well (known to the  verifier and prover). 


First we import the necessary dependencies and set up logging to be as informative as possible. 

In [ ]:
from torch import nn
import ezkl
import os
import json
import logging

# uncomment for more descriptive logging 
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.INFO)


Now we define our model. It is a humble model with but a conv layer and a $ReLU$ non-linearity, but it is a model nonetheless

In [ ]:
import torch
# Defines the model
# we got convs, we got relu, 
# What else could one want ????

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=5, stride=4)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)

        return x


circuit = MyModel()

# this is where you'd train your model




We omit training for purposes of this demonstration. We've marked where training would happen in the cell above. 
Now we export the model to onnx and create a corresponding (randomly generated) input file.

You can replace the random `x` with real data if you so wish. 

In [ ]:
x = torch.rand(1,*[3, 8, 8], requires_grad=True)

# Flips the neural net into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                      x,                   # model input (or a tuple for multiple inputs)
                      "network.onnx",            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize data into file:
json.dump( data, open("input.json", 'w' ))



================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



This is where the magic happens. We define our `PyRunArgs` objects which contains the visibility parameters for out model. 
- `input_visibility` defines the visibility of the model inputs
- `param_visibility` defines the visibility of the model weights and constants and parameters 
- `output_visibility` defines the visibility of the model outputs

There are currently 4 visibility settings:
- `public`: known to both the verifier and prover (a subtle nuance is that this may not be the case for model parameters but until we have more rigorous theoretical results we don't want to make strong claims as to this). 
- `private`: known only to the prover
- `hashed`: the hash pre-image is known to the prover, the prover and verifier know the hash. The prover proves that the they know the pre-image to the hash. 
- `encrypted`: the non-encrypted element and the secret key used for decryption are known to the prover. The prover and the verifier know the encrypted element, the public key used to encrypt, and the hash of the decryption hey. The prover proves that they know the pre-image of the hashed decryption key and that this key can in fact decrypt the encrypted message.

Here we create the following setup:
- `input_visibility`: "hashed"
- `param_visibility`: "hashed"
- `output_visibility`: public

We encourage you to play around with other setups :) 

Shoutouts: 

- [summa-solvency](https://github.com/summa-dev/summa-solvency) for their help with the poseidon hashing chip. 
- [timeofey](https://github.com/timoftime) for providing inspiration in our developement of the el-gamal encryption circuit in Halo2. 

In [ ]:
import ezkl

model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')
srs_path = os.path.join('kzg.srs')
data_path = os.path.join('input.json')

run_args = ezkl.PyRunArgs()
run_args.input_visibility = "hashed"
run_args.param_visibility = "hashed"
run_args.output_visibility = "public"





Now we generate a settings file. This file basically instantiates a bunch of parameters that determine their circuit shape, size etc... Because of the way we represent nonlinearities in the circuit (using Halo2's [lookup tables](https://zcash.github.io/halo2/design/proving-system/lookup.html)), it is often best to _calibrate_ this settings file as some data can fall out of range of these lookups.

You can pass a dataset for calibration that will be representative of real inputs you might find if and when you deploy the prover. Here we create a dummy calibration dataset for demonstration purposes. 

In [ ]:
!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path, py_run_args=run_args)
assert res == True

INFO ezkl.graph.model 2023-07-21 17:15:24,920 model.rs:421 set batch size to 1
INFO ezkl.graph.model 2023-07-21 17:15:24,923 model.rs:746 calculating num of constraints using dummy model layout...
INFO ezkl.graph.model 2023-07-21 17:15:24,924 model.rs:246 model generates 78 constraints (excluding modules)


In [ ]:
# generate a bunch of dummy calibration data
cal_data = {
    "input_data": [torch.cat((x, torch.rand(10, *[3, 8, 8]))).flatten().tolist()],
}

cal_path = os.path.join('val_data.json')
# save as json file
with open(cal_path, "w") as f:
    json.dump(cal_data, f)

res = await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources")

TypeError: cat() received an invalid combination of arguments - got (Tensor, Tensor), but expected one of:
 * (tuple of Tensors tensors, int dim, *, Tensor out)
 * (tuple of Tensors tensors, name dim, *, Tensor out)


In [ ]:
res = ezkl.compile_model(model_path, compiled_model_path, settings_path)
assert res == True

As we use Halo2 with KZG-commitments we need an SRS string from (preferably) a multi-party trusted setup ceremony. For an overview of the procedures for such a ceremony check out [this page](https://blog.ethereum.org/2023/01/16/announcing-kzg-ceremony). The `get_srs` command retrieves a correctly sized SRS given the calibrated settings file from [here](https://github.com/han0110/halo2-kzg-srs). 

These SRS were generated with [this](https://github.com/privacy-scaling-explorations/perpetualpowersoftau) ceremony. 

In [37]:
res = ezkl.get_srs(srs_path, settings_path)


INFO ezkl.execute 2023-07-21 17:15:30,021 execute.rs:502 SRS downloaded


We now need to generate the (partial) circuit witness. These are the model outputs (and any hashes) that are generated when feeding the previously generated `input.json` through the circuit / model. 

In [ ]:
!export RUST_BACKTRACE=1

witness_path = "witness.json"

res = ezkl.gen_witness(data_path, compiled_model_path, witness_path, settings_path = settings_path)

As a sanity check you can "mock prove" (i.e check that all the constraints of the circuit match without generate a full proof). 

In [47]:


res = ezkl.mock(witness_path, compiled_model_path, settings_path)

INFO ezkl.graph.model 2023-07-21 17:16:10,233 model.rs:421 set batch size to 1
INFO ezkl.graph 2023-07-21 17:16:10,234 mod.rs:560 public inputs lengths: [1]
INFO ezkl.execute 2023-07-21 17:16:10,235 execute.rs:777 Mock proof
INFO ezkl.graph.model 2023-07-21 17:16:10,235 model.rs:572 configuring model
INFO ezkl.graph.model 2023-07-21 17:16:10,643 model.rs:604 model layout...
INFO ezkl.graph.model 2023-07-21 17:16:10,709 model.rs:666 computing...
INFO ezkl.graph.model 2023-07-21 17:16:10,710 model.rs:666 computing...


Here we setup verifying and proving keys for the circuit. As the name suggests the proving key is needed for ... proving and the verifying key is needed for ... verifying. 

In [40]:
# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        srs_path,
        settings_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

INFO ezkl.graph.model 2023-07-21 17:15:30,929 model.rs:421 set batch size to 1
INFO ezkl.pfsys.srs 2023-07-21 17:15:30,930 srs.rs:23 loading srs from "kzg.srs"
INFO ezkl.execute 2023-07-21 17:15:30,935 execute.rs:1634 downsizing params to 15 logrows
INFO ezkl.graph.model 2023-07-21 17:15:30,935 model.rs:572 configuring model
INFO ezkl.graph.model 2023-07-21 17:15:30,961 model.rs:604 model layout...
INFO ezkl.graph.model 2023-07-21 17:15:30,978 model.rs:666 computing...
INFO ezkl.graph.model 2023-07-21 17:15:30,979 model.rs:666 computing...
INFO ezkl.pfsys 2023-07-21 17:15:32,265 mod.rs:369 VK took 1.330
INFO ezkl.graph.model 2023-07-21 17:15:32,266 model.rs:572 configuring model
INFO ezkl.graph.model 2023-07-21 17:15:32,291 model.rs:604 model layout...
INFO ezkl.graph.model 2023-07-21 17:15:32,307 model.rs:666 computing...
INFO ezkl.graph.model 2023-07-21 17:15:32,308 model.rs:666 computing...
INFO ezkl.pfsys 2023-07-21 17:15:33,164 mod.rs:375 PK took 0.898
INFO ezkl.pfsys 2023-07-21 1

Now we generate a full proof. 

In [41]:
# GENERATE A PROOF

proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        srs_path,
        "evm",
        "single",
        settings_path,
    )

print(res)
assert os.path.isfile(proof_path)

INFO ezkl.graph.model 2023-07-21 17:15:33,383 model.rs:421 set batch size to 1
INFO ezkl.graph 2023-07-21 17:15:33,384 mod.rs:560 public inputs lengths: [1]
INFO ezkl.pfsys.srs 2023-07-21 17:15:33,385 srs.rs:23 loading srs from "kzg.srs"
INFO ezkl.execute 2023-07-21 17:15:33,389 execute.rs:1634 downsizing params to 15 logrows
INFO ezkl.pfsys 2023-07-21 17:15:33,390 mod.rs:519 loading proving key from "test.pk"
INFO ezkl.graph.model 2023-07-21 17:15:33,390 model.rs:572 configuring model
INFO ezkl.pfsys 2023-07-21 17:15:33,635 mod.rs:429 proof started...
INFO ezkl.graph.model 2023-07-21 17:15:33,635 model.rs:572 configuring model
INFO ezkl.graph.model 2023-07-21 17:15:34,041 model.rs:604 model layout...
INFO ezkl.graph.model 2023-07-21 17:15:34,056 model.rs:666 computing...
INFO ezkl.graph.model 2023-07-21 17:15:34,057 model.rs:666 computing...
INFO ezkl.execute 2023-07-21 17:15:36,555 execute.rs:1177 proof took 2.920


{'instances': [[[0, 0, 0, 0]], [[14274608303896394872, 3702323036926418149, 7698009275440366899, 3432088659181290681], [0, 0, 0, 0]]], 'proof': '13c139f3d08f72dfd2c174e8471069c3bf1c2c3dec87dd77b9ead4cefc56c50509d840547d4aa6d16bea4be2f6b38675400b6c6731bb30b3effb8ada4a669ee31c9d01379818415b6ef3f62c76674f8b9766b8e3205a628944dfd93f6a03f9df25fed4a60e68946d9427be5fa26448ad5e9970eb6110e6b24adfa4b5f806651e11f8f29c7dcbe2241493779fa0d6c83b43fa69f60af6cdd909d2769abba44d4f137797013a87a07e9e0c7afded2f25ffc951cc084a81c46f1011cef335358927123caa38f6dc4c4062248a8f08a67162761513fddf4217a4961ee7418812b5a105477b81794920ab0ac98571d246948546feb06332f08b03bbb7de8730c025d430297460d6e49b9b693a97ff720dab9af6dbdb988a8acef588a4a895f0ca123e07b8e041e29f28d5ff7871d21da0f2553a03f4e601f439c9c61c404e4754316a27a9fa32dc3ebb2f96d75df938cbfffb21d102c3558b3af07b4d217f6cc7733e20a26dc69e086c5c8d2fe16881c08d3d12bea7588a69410152e7d4bf1a453b280673294a1d39d99b0b21b1362aa651e6a6ac72e8ecf4fee63e491c998d04dc800a4a828fcad298ea337d0a9

And verify it as a sanity check. 

In [42]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        srs_path,
    )

assert res == True
print("verified")

INFO ezkl.pfsys.srs 2023-07-21 17:15:36,571 srs.rs:23 loading srs from "kzg.srs"
INFO ezkl.execute 2023-07-21 17:15:36,576 execute.rs:1634 downsizing params to 15 logrows
INFO ezkl.pfsys 2023-07-21 17:15:36,578 mod.rs:498 loading verification key from "test.vk"
INFO ezkl.graph.model 2023-07-21 17:15:36,579 model.rs:572 configuring model
INFO ezkl.execute 2023-07-21 17:15:36,594 execute.rs:1593 verify took 0.8
INFO ezkl.execute 2023-07-21 17:15:36,594 execute.rs:1598 verified: true


verified


We can now create an EVM / `.sol` verifier that can be deployed on chain to verify submitted proofs using a view function.

In [43]:

abi_path = 'test.abi'
sol_code_path = 'test.sol'

res = ezkl.create_evm_verifier(
        vk_path,
        srs_path,
        settings_path,
        sol_code_path,
        abi_path,
    )
assert res == True


INFO ezkl.execute 2023-07-21 17:15:36,600 execute.rs:80 checking solc installation..
INFO ezkl.pfsys.srs 2023-07-21 17:15:36,601 srs.rs:23 loading srs from "kzg.srs"
INFO ezkl.execute 2023-07-21 17:15:36,604 execute.rs:1634 downsizing params to 15 logrows
INFO ezkl.pfsys 2023-07-21 17:15:36,605 mod.rs:498 loading verification key from "test.vk"
INFO ezkl.graph.model 2023-07-21 17:15:36,605 model.rs:572 configuring model


## Verify on the evm

In [44]:
# Make sure anvil is running locally first
# run with $ anvil -p 3030
# we use the default anvil node here
import json

address_path = os.path.join("address.json")

res = ezkl.deploy_evm(
    address_path,
    sol_code_path,
    'http://127.0.0.1:3030'
)

assert res == True

with open(address_path, 'r') as file:
    addr = file.read().rstrip()

INFO ezkl.execute 2023-07-21 17:15:36,914 execute.rs:80 checking solc installation..


RuntimeError: Failed to run deploy_evm: error sending request for url (http://127.0.0.1:3030/): error trying to connect: tcp connect error: Connection refused (os error 61)

In [ ]:
# make sure anvil is running locally
# $ anvil -p 3030

res = ezkl.verify_evm(
    proof_path,
    addr,
    "http://127.0.0.1:3030"
)
assert res == True